## Import some needed scripts

In [1]:
from database import Database
from gatherData import getDataFromTweeter
from ml import getFeatures
from pandas import DataFrame
from pprint import pprint

## Define some constants needed

In [2]:
MONGO_DB_URI = "mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&directConnection=true&ssl=false"
USER_DATASET_FILE = "users_dataset.csv"

## Gather data using twitter API and populate MongoDB

We gather the data in a way to try to maintain a 50-50% of bots and humans

In [ ]:
getDataFromTweeter(MONGO_DB_URI,USER_DATASET_FILE)

In [3]:
dataset = getFeatures(MONGO_DB_URI)
dataset.to_csv("dataset.csv")
pprint(dataset.head())

User 21630333 (BOT) has no tweets in the database
User 22036045 (HUMAN) has no tweets in the database
User 22730529 (HUMAN) has no tweets in the database
User 23220415 (BOT) has no tweets in the database
User 23223395 (HUMAN) has no tweets in the database
User 23222867 (BOT) has no tweets in the database
User 23224336 (BOT) has no tweets in the database
User 23258087 (BOT) has no tweets in the database
User 23258968 (BOT) has no tweets in the database
User 25070587 (HUMAN) has no tweets in the database
User 23272942 (BOT) has no tweets in the database
User 24221453 (BOT) has no tweets in the database
User 31626280 (HUMAN) has no tweets in the database
User 25008627 (BOT) has no tweets in the database
User 25292754 (BOT) has no tweets in the database
User 33114252 (BOT) has no tweets in the database
User 37193429 (BOT) has no tweets in the database
User 37967468 (BOT) has no tweets in the database
User 43624887 (BOT) has no tweets in the database
User 49548467 (BOT) has no tweets in the

TypeError: 'module' object is not callable